In [1]:
#函數有更新的時候會自動重新載入函數的指令
%load_ext autoreload
%autoreload 2
#-------------------------------------
import os
from dotenv import load_dotenv
import pandas as pd
import random
#from API.FileReader import readkl, readCsv, readGeneratedRegex
from API.FileProcessor import merge_dataframes_to_dict, prompt_generator, get_grouped_sid_to_regex_dict, prompt_generator_no_random
from API.regexEvaluation import positive_evaluation, negative_evaluation
from API.Drawer import draw_outcome_histogram, draw_outcome_bar
from API.Drawer import draw_positive_contribution_of_generated_regex, draw_negative_contribution_of_generated_regex
from share_data import sharedData
from API.FileReader import read_pkl, read_csv

### 找出SID有幾個unique text還有占比

In [ ]:
unique_text_counts = imap_df.groupby('sid')['text'].nunique()

# Filter to include only those 'sid' with five or more unique 'text' values
filtered_sids = unique_text_counts[unique_text_counts >= 5].index

# Filter the original DataFrame to include only the rows with the filtered 'sid' values
filtered_imap_df = imap_df[imap_df['sid'].isin(filtered_sids)]

# Count the total number of 'text' occurrences for each filtered 'sid'
sid_text_counts = filtered_imap_df.groupby('sid').size().reset_index(name='text_count')

# Calculate the total number of 'text' occurrences in the entire dataset
total_texts = imap_df.shape[0]

for sid, text_count in zip(sid_text_counts['sid'], sid_text_counts['text_count']):
    print(f"SID: {sid}, Percentage: {text_count/total_texts}")

# Display the results
print(sid_text_counts)
print(f"Total number of 'text' occurrences in the dataset: {total_texts}")

In [ ]:
import pandas as pd

# Load the dataset
#imap_df = pd.read_csv('imap.csv')

# Group by 'sid' and count unique texts per 'sid'
#unique_text_counts = imap_df.groupby('sid')['text'].nunique()

# Filter 'sids' with 5 or more unique texts
#filtered_sids = unique_text_counts[unique_text_counts >= 5].index

# Create a DataFrame that includes only filtered 'sids'
#filtered_imap_df = imap_df[imap_df['sid'].isin(filtered_sids)]

# Count total occurrences of 'text' for each 'sid' in the filtered DataFrame
sid_text_counts = imap_df.groupby('sid').size().reset_index(name='text_count')

# Calculate the total number of 'text' occurrences in the entire dataset
total_texts = imap_df.shape[0]

# Iterate through each 'sid' in the filtered list and print the percentage of text occurrences
for sid, text_count in zip(sid_text_counts['sid'], sid_text_counts['text_count']):
    print(f"SID: {sid}, Percentage: {text_count / total_texts * 100:.2f}%")

# Print the count data
print(sid_text_counts)

print(f"Total number of 'text' occurrences in the dataset: {total_texts}")

In [ ]:
# Group by 'sid' and count occurrences
sid_text_counts = imap_df.groupby('sid').size().reset_index(name='text_count')

# Calculate the total number of 'text' occurrences in the entire dataset
total_texts = imap_df.shape[0]

# Calculate unique text counts per sid
unique_text_counts = imap_df.groupby('sid')['text'].nunique().reset_index(name='unique_text_count')

# Join unique text counts to sid text counts DataFrame
sid_text_counts = sid_text_counts.merge(unique_text_counts, on='sid')

# Calculate percentage for better sorting
sid_text_counts['Percentage'] = sid_text_counts['text_count'] / total_texts * 100

# # Sort by 'sid'
# sorted_by_sid = sid_text_counts.sort_values(by='sid')
# print("Sorted by SID:")
# print(sorted_by_sid)

# Sort by 'Percentage' of text occurrences
sorted_by_percentage = sid_text_counts.sort_values(by='Percentage', ascending=False)
print("\nSorted by Percentage of Text Occurrences:")
print(sorted_by_percentage)

# # Sort by 'unique_text_count'
# sorted_by_unique_texts = sid_text_counts.sort_values(by='unique_text_count', ascending=False)
# print("\nSorted by Number of Unique Texts:")
# print(sorted_by_unique_texts)

# Print the total number of 'text' occurrences in the dataset
#print(f"\nTotal number of 'text' occurrences in the dataset: {total_texts}")
print()
num_sid_zero_unique_texts = (sid_text_counts['unique_text_count'] == 0).sum()
print(f"Number of SIDs with zero unique texts: {num_sid_zero_unique_texts}")
total_sids = imap_df['sid'].nunique()
print(f"Total number of unique SIDs: {total_sids}")
total_sids = len(imap_df['sid'])
num_rows_with_no_text = imap_df['text'].isnull().sum()
print('imap_df')
print(f"Total number of binary (including duplicates): {num_rows_with_no_text}")
print(f"Total number of SIDs (including duplicates): {total_sids}")
print(f"{num_rows_with_no_text/total_sids*100:.2f}% of rows have no text")

In [ ]:
import os
import pandas as pd

# Ensure the directory exists
os.makedirs('./df_data', exist_ok=True)

# List of DataFrames to iterate through
dataframes = {
    'pop_df': pop_df,
    'imap_df': imap_df,
    'smtp_df': smtp_df,
    'sip_df': sip_df
}

# Process each DataFrame
for name, df in dataframes.items():
    # Group by 'sid' and count occurrences
    sid_text_counts = df.groupby('sid').size().reset_index(name='text_count')

    # Calculate the total number of 'text' occurrences in the entire dataset
    total_texts = df.shape[0]

    # Calculate unique text counts per sid
    unique_text_counts = df.groupby('sid')['text'].nunique().reset_index(name='unique_text_count')

    # Join unique text counts to sid text counts DataFrame
    sid_text_counts = sid_text_counts.merge(unique_text_counts, on='sid')

    # Calculate percentage for better sorting
    sid_text_counts['Percentage'] = sid_text_counts['text_count'] / total_texts * 100

    # Sort by 'Percentage' of text occurrences
    sorted_by_percentage = sid_text_counts.sort_values(by='Percentage', ascending=False)

    # Prepare the results to be written to the text file
    result = []
    result.append(f"\nSorted by Percentage of Text Occurrences ({name}):")
    result.append(sorted_by_percentage.to_string(index=False))
    num_sid_zero_unique_texts = (sid_text_counts['unique_text_count'] == 0).sum()
    total_sids = df['sid'].nunique()
    num_rows_with_no_text = df['text'].isnull().sum()
    total_sids_including_duplicates = len(df['sid'])

    result.append(f"\nNumber of SIDs with zero unique texts: {num_sid_zero_unique_texts}")
    result.append(f"Total number of unique SIDs: {total_sids}")
    result.append(f"Total number of SIDs (including duplicates): {total_sids_including_duplicates}")
    result.append(f"Total number of binary (including duplicates): {num_rows_with_no_text}")
    result.append(f"{num_rows_with_no_text/total_sids_including_duplicates*100:.2f}% of rows have no text")

    # Identify SIDs with 5 or more unique texts
    sids_with_5_or_more_unique_texts = unique_text_counts[unique_text_counts['unique_text_count'] >= 5]

    result.append(f"\nSIDs with 5 or more unique texts ({name}):")
    result.append(sids_with_5_or_more_unique_texts.to_string(index=False))

    # Write the results to a text file
    with open(f'./df_data/{name}_df.txt', 'w') as file:
        file.write('\n'.join(result))

print("Processing complete. Results stored in ./df_data/ directory.")


### 讀取資料集

In [ ]:
pop_df = read_pkl("dataset/pop_report_with_tknscore_new.pkl")
imap_df = read_pkl("dataset/imap_report_with_tknscore_new.pkl")
smtp_df = read_pkl("dataset/smtp_report_with_tknscore_new.pkl")
sip_df = read_pkl("dataset/sip_report_with_tknscore_new.pkl")
label_df = read_csv("dataset/sid_table(packet).csv")

In [12]:
directory_path = r"C:\NCU\Intern\AutomaticNetwork IntrusionRuleGeneration\experiment\1 out of 9\generated regex"

shared_data = sharedData(directory_path)

In [ ]:
unfiltered_sids_to_unique_texts: pd.DataFrame = shared_data.pop_df.groupby('sid')['text'].apply(lambda x: list(set(x.dropna()))).reset_index(name='texts')
filtered_sids_to_unique_texts: pd.DataFrame = unfiltered_sids_to_unique_texts[unfiltered_sids_to_unique_texts['texts'].apply(lambda x: len(x) > 0)]

print(type(filtered_sids_to_unique_texts))

#for sid, texts in zip(filtered_sids_to_unique_texts['sid'], filtered_sids_to_unique_texts['texts']):
#    print(texts)

### 將所有資料集轉換為dictionary

In [ ]:
dfs = [pop_df, imap_df, smtp_df, sip_df]
sid_to_unique_texts_dict = merge_dataframes_to_dict(dfs)
sid_to_unique_texts_dict

### 實驗參數初始化


In [ ]:
NUMBER_OF_ONE_REGEX_GROUP = 9
EXPERIMENT_NAME = "1 out of 9"
PROCESS_TYPE = "with post-process" #with/without post-process
MODE = "fraction" #fraction/integer
THRESHOLD = 2/3
#escaped_string = original_string.replace('"', '\\"')
if not os.path.exists(f"experiment/{EXPERIMENT_NAME}"):
    os.makedirs(f"experiment/{EXPERIMENT_NAME}")

#讀取csv 每個row是一個regex begin
#generated_regex_list: list = readGeneratedRegex(f"experiment/{EXPERIMENT_NAME}/generated regex.csv")
#grouped_sid_to_regex_dict = get_grouped_sid_to_regex_dict(generated_regex_list, 9, sid_to_unique_texts_dict)
#讀取csv 每個row是一個regex end

#讀取csv 每個row是一個dict+後處理 begin
#generated_regex_list = cleanAllNewlineinList(generated_regex_list)
#sid_to_unique_texts_dict = cleanAllNewlineinDict(sid_to_unique_texts_dict)
#grouped_sid_to_regex_dict = merged_cleaned_regex_dictionary
#讀取csv 每個row是一個dict+後處理 end

In [ ]:
import regex
p = 'DELE 97'
bool(regex.search(pattern, target_text, regex.DOTALL))

In [13]:
import os

# Set the directory containing the text files
directory_path = r"C:\NCU\Intern\AutomaticNetwork IntrusionRuleGeneration\experiment\1 out of 9\generated regex"

# # Dictionary to hold the file contents
# file_contents = {}

# # Read each file in the directory and store its contents in the dictionary
# for filename in os.listdir(directory_path):
#     file_path = os.path.join(directory_path, filename)
#     with open(file_path, 'r', encoding='utf-8') as file:
#         file_contents[filename] = file.readlines()

# # Remove the '.txt' extension from the keys in the dictionary
# file_contents_no_ext = {filename[:-4]: contents for filename, contents in file_contents.items()}

# # Print the dictionary
# print(file_contents_no_ext)
print(shared_data.get_generated_regex().head())

       sid                                              regex
0  1161912  [^DELE \d{1,3}\r\n$, ^DELE [1-9][0-9]{0,2}\r\n...
1  1176066  [^(RETR \d{1,3}\r\n)$, ^(TOP \d{1,3} 0\r\n)$, ...
2  1239754  [^GET http:\/\/clientapi\.ipip\.net\/echo\.php...
3  1454621  [^EHLO\s[\w\.-]+\r\n$, ^EHLO\s(\d{1,3}\.){3}\d...
4  1491662  [^(POST /ipp HTTP/1\.1\r\nHost: [0-9]+\.[0-9]+...


In [ ]:
print(grouped_sid_to_regex_dict)
print(list(grouped_sid_to_regex_dict.items()))

### 開檔案

In [ ]:
directory = f'./experiment/{EXPERIMENT_NAME}/graph/{PROCESS_TYPE}'
if not os.path.exists(directory):
    os.makedirs(directory)

### prompt生成

In [ ]:
prompt_generator_no_random(sid_to_unique_texts_dict, EXPERIMENT_NAME, 6)

### 執行正樣本比對與負樣本拒絕測試
- 將結果儲存於./experiment/{EXPERIMENT_NAME}/data/{PROCESS_TYPE}/sid.txt 
- 計算positive_sid_to_success_rate_dict, positive_text_to_error_regex_dict，negative_sid_to_seccess_rate_dict，negative_text_to_error_regex_dict用於後續的作圖

In [ ]:
import regex
positive_sid_to_success_rate_dict = {}
negative_sid_to_success_rate_dict = {}
positive_text_to_error_regex_dict = {}
negative_text_to_error_regex_dict = {}
directory = f'./experiment/{EXPERIMENT_NAME}/data/{PROCESS_TYPE}'
if not os.path.exists(directory):
    os.makedirs(directory)
# post sementic process
for sid, regex_list in grouped_sid_to_regex_dict.items():
    transformed_data_step1 = [entry.rstrip('$\n') for entry in grouped_sid_to_regex_dict[sid]]
    transformed_data_step2 = [entry.replace('\\r\\n', '\r\n') for entry in transformed_data_step1]
    grouped_sid_to_regex_dict[sid] = transformed_data_step2
for sid, regex_list in grouped_sid_to_regex_dict.items():   
    with open(f'{directory}/{sid}.txt', 'w') as file:
        positive_evaluation(regex_list, sid, sid_to_unique_texts_dict, file, positive_sid_to_success_rate_dict, positive_text_to_error_regex_dict, THRESHOLD ,MODE)
        negative_evaluation(regex_list, sid, sid_to_unique_texts_dict, file, negative_sid_to_success_rate_dict, negative_text_to_error_regex_dict, THRESHOLD, MODE)

# transformed_data_step1 = [entry.rstrip('$\n') for entry in grouped_sid_to_regex_dict['1161912']]
# transformed_data_step2 = [entry.replace('\\r\\n', '\r\n') for entry in transformed_data_step1]
# grouped_sid_to_regex_dict['1161912'] = transformed_data_step2
# with open(f'{directory}/{1161912}.txt', 'w') as file:
#     positive_evaluation(transformed_data_step2, '1161912', sid_to_unique_texts_dict, file, positive_sid_to_success_rate_dict, positive_text_to_error_regex_dict, THRESHOLD ,MODE)
#     negative_evaluation(grouped_sid_to_regex_dict['1161912'], '1161912', sid_to_unique_texts_dict, file, negative_sid_to_success_rate_dict, negative_text_to_error_regex_dict, THRESHOLD, MODE)
#後處理 "前面加\
#把首尾的/去掉\

### 計算貢獻度


In [ ]:
# 建立 sid_to_unique_texts_dict
#sid_to_unique_texts_dict = {sid: set(texts) for sid, texts in grouped_sid_to_regex_dict.items()}
# 新的結果字典
positive_expand_dict = draw_positive_contribution_of_generated_regex(grouped_sid_to_regex_dict, sid_to_unique_texts_dict, positive_text_to_error_regex_dict, EXPERIMENT_NAME)
negative_expand_dict = draw_negative_contribution_of_generated_regex(grouped_sid_to_regex_dict, sid_to_unique_texts_dict, negative_text_to_error_regex_dict, EXPERIMENT_NAME)

In [ ]:
import json
def get_hiest_score_dict(expand_dict):
    result = {}
    for key, value in expand_dict.items():
        text_list = value['text list']
        max_text = max(text_list, key=text_list.get)
        max_score = text_list[max_text]
        result[key] = {'text': max_text, 'score': max_score}
    return result

positive_highest_score_dict = get_hiest_score_dict(positive_expand_dict)
negative_highest_score_dict = get_hiest_score_dict(negative_expand_dict)

# Write positive_expand_dict in a human-readable format
with open(f'./experiment/{EXPERIMENT_NAME}/data/{PROCESS_TYPE}/positive_expand_dict.txt', 'w') as file:
    json.dump(positive_highest_score_dict, file, indent=4)

# Write negative_expand_dict in a human-readable format
with open(f'./experiment/{EXPERIMENT_NAME}/data/{PROCESS_TYPE}/negative_expand_dict.txt', 'w') as file:
    json.dump(negative_highest_score_dict, file, indent=4)
    
import matplotlib.pyplot as plt

def plot_highest_scores(highest_score_dict, title, save_path):
    ids = list(highest_score_dict.keys())
    scores = [highest_score_dict[id]["score"] for id in ids]

    plt.figure(figsize=(12, 6))
    bars = plt.bar(ids, scores, color='skyblue')
    plt.xlabel('ID Key')
    plt.ylabel('Score')
    plt.title(title)
    plt.ylim(0, 1.1)  # Limit y-axis to 110% to provide space for annotations
    plt.xticks(rotation=45)

    # Annotate bars with percentage values
    for bar in bars:
        height = bar.get_height()
        plt.annotate(f'{height*100:.1f}%',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom',
                     rotation=45)  # Rotate the text to prevent overlap

    # Add padding to the top of the plot to prevent title overlap
    plt.subplots_adjust(top=0.85)

    # Save the plot
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f'{title}.png'))

    plt.show()
    
plot_highest_scores(positive_highest_score_dict, f'Positive Highest Scores {PROCESS_TYPE}', save_path=f'./experiment/{EXPERIMENT_NAME}/graph/{PROCESS_TYPE}')
plot_highest_scores(negative_highest_score_dict, f'Negative Highest Scores {PROCESS_TYPE}', save_path=f'./experiment/{EXPERIMENT_NAME}/graph/{PROCESS_TYPE}')

In [ ]:
# base_path = os.path.join('./experiment', 'test', 'graph/contribution/positive')
# os.makedirs(base_path, exist_ok=True)

In [ ]:
negative_sid_to_success_rate_dict

In [ ]:
draw_outcome_bar(positive_sid_to_success_rate_dict,EXPERIMENT_NAME, f"Positive Accuracy {PROCESS_TYPE}")
draw_outcome_histogram(positive_sid_to_success_rate_dict, EXPERIMENT_NAME, f"Distribution of Positive Success Rates {PROCESS_TYPE}")
draw_outcome_bar(negative_sid_to_success_rate_dict,EXPERIMENT_NAME, f"Negative Accuracy {PROCESS_TYPE}")
draw_outcome_histogram(negative_sid_to_success_rate_dict, EXPERIMENT_NAME, f"Distribution of Negative Success Rates {PROCESS_TYPE}")

In [ ]:
import google.generativeai as genai
load_dotenv()

system_prompt = '''
Please find a regular expression to match all packet payloads.
You need to find the similarities in the sentences and generalize the parts where they differ. 
The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. 
There will be examples to help you find the patterns. 
[‘DELE 3\r\n’, ‘DELE 128\r\n’, ‘DELE 74\r\n’, ‘DELE 22\r\n’, ‘DELE 70\r\n’] 
These examples show the attacker is trying to delete someone’s email by POP protocol. 
The index of the desired mail is indicated under the DELE command. 
Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\r\n)$’ 

With the given example payloads: 
[‘EHLO BtuCBHdSb51.com\r\n’, ‘EHLO 203.187.87.27\r\n’, ‘EHLO slae02Fo9Ep.com\r\n’, ‘EHLO 210.64.37.51\r\n’, ‘EHLO LLb0RwqdbkikFWo.com\r\n’] 
These examples show the attacker is trying to make sure the SMTP server is up and running. The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. 
Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\r\n)$’ 

note: Your output must be a in jason format，no need to write Markdown
'''
user_prompt = '''
['DELE 107\r\n', 'DELE 124\r\n', 'DELE 113\r\n', 'DELE 7\r\n', 'DELE 105\r\n', 'DELE 71\r\n', 'DELE 122\r\n', 'DELE 25\r\n', 'DELE 61\r\n', 'DELE 100\r\n', 'DELE 65\r\n', 'DELE 52\r\n', 'DELE 69\r\n', 'DELE 43\r\n', 'DELE 21\r\n', 'DELE 67\r\n', 'DELE 90\r\n', 'DELE 14\r\n', 'DELE 109\r\n', 'DELE 77\r\n', 'DELE 73\r\n', 'DELE 104\r\n', 'DELE 45\r\n', 'DELE 10\r\n', 'DELE 89\r\n', 'DELE 3\r\n', 'DELE 120\r\n', 'DELE 96\r\n', 'DELE 48\r\n', 'DELE 95\r\n', 'DELE 110\r\n', 'DELE 12\r\n', 'DELE 27\r\n', 'DELE 56\r\n', 'DELE 34\r\n', 'DELE 55\r\n', 'DELE 132\r\n', 'DELE 38\r\n', 'DELE 26\r\n', 'DELE 86\r\n', 'DELE 115\r\n', 'DELE 103\r\n', 'DELE 17\r\n', 'DELE 101\r\n', 'DELE 8\r\n', 'DELE 11\r\n', 'DELE 33\r\n', 'DELE 53\r\n', 'DELE 88\r\n', 'DELE 76\r\n']
Please give 3 possible and different regular expressions to match all of the elements. 
You can give only 1 expression if the 3 expressions you find are too similar. 
Let’s work this out in a step-by-step way to make sure we have the right answer. 
To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\r\n’, ‘CAPA\r\n’, ‘\x15\x03\x01’, ‘GET / HTTP/1.0\r\n\r\n’, ‘r\n\r\n’] 
You only need to give me the three regular expressions in code format.
'''
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt,
)

response = model.generate_content(user_prompt)
print(response.text)

In [ ]:
### Gmni
load_dotenv()
import google.generativeai as genai
system_prompt = "You are a professional Regular Expression maker."

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash"
)

### OLD CODE

In [ ]:
#獲取各個SID所對應到的text
def map_sid_to_unique_texts(df):
    sid_to_texts = {}
    for _, row in df.iterrows():
        sid = row['sid']
        text = row['text']
        if pd.notnull(text):  # Ensure text is not NaN
            if sid in sid_to_texts:
                sid_to_texts[sid].add(text)
            else:
                sid_to_texts[sid] = {text}
    return sid_to_texts
sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
#print(sid_to_unique_texts)

In [ ]:
#過濾掉text長度小於5的sid
def filter_sids_by_text_length(sit_to_text_dict):
    filtered_sids = set()
    for key, value in sit_to_text_dict.items():
        if len(value) > 5:  # Convert to string to avoid errors with non-string types
            filtered_sids.add(key)
    return filtered_sids

# Assuming pop_df is your DataFrame
filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
#print(filtered_sids)
#print(len(filtered_sids))

In [ ]:
def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                print (row['pcre'])
                return row['pcre'].strip('/')
    return None

# Iterate Throush Positive payload text to see if the ans regex is correct    
def is_ans_correct(sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + bool(regex.search(ansPattern, text,regex.DOTALL))
        if not bool(regex.search(r'/^(GET|HEAD|POST|OPTIONS)( )(\/|\/version|\/api.*|\/jsproxy)( )(HTTP\/1\.(?:0|1))(\r\n)(.*)$/', text,regex.DOTALL)):
            errorList.append(text)

    print("answer test")
    print(f"correct: {correct}, total: {total}")
    print(len(errorList))
    for error in errorList:
        print(f"error: {error}")
    return correct, total
is_ans_correct('1783777')

### ChatGPT API


In [ ]:
load_dotenv()
CHAT_GPT_API_KEY = os.getenv('C_API_KEY')
client = OpenAI(
    api_key=CHAT_GPT_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


### Gemini API

In [ ]:
import google.generativeai as genai
load_dotenv()

system_prompt = '''
Please find a regular expression to match all packet payloads.
You need to find the similarities in the sentences and generalize the parts where they differ. 
The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. 
There will be examples to help you find the patterns. 
[‘DELE 3\r\n’, ‘DELE 128\r\n’, ‘DELE 74\r\n’, ‘DELE 22\r\n’, ‘DELE 70\r\n’] 
These examples show the attacker is trying to delete someone’s email by POP protocol. 
The index of the desired mail is indicated under the DELE command. 
Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\r\n)$’ 

With the given example payloads: 
[‘EHLO BtuCBHdSb51.com\r\n’, ‘EHLO 203.187.87.27\r\n’, ‘EHLO slae02Fo9Ep.com\r\n’, ‘EHLO 210.64.37.51\r\n’, ‘EHLO LLb0RwqdbkikFWo.com\r\n’] 
These examples show the attacker is trying to make sure the SMTP server is up and running. The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. 
Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\r\n)$’ 

note: Your output must be a in jason format，no need to write Markdown
'''
user_prompt = '''
['DELE 107\r\n', 'DELE 124\r\n', 'DELE 113\r\n', 'DELE 7\r\n', 'DELE 105\r\n', 'DELE 71\r\n', 'DELE 122\r\n', 'DELE 25\r\n', 'DELE 61\r\n', 'DELE 100\r\n', 'DELE 65\r\n', 'DELE 52\r\n', 'DELE 69\r\n', 'DELE 43\r\n', 'DELE 21\r\n', 'DELE 67\r\n', 'DELE 90\r\n', 'DELE 14\r\n', 'DELE 109\r\n', 'DELE 77\r\n', 'DELE 73\r\n', 'DELE 104\r\n', 'DELE 45\r\n', 'DELE 10\r\n', 'DELE 89\r\n', 'DELE 3\r\n', 'DELE 120\r\n', 'DELE 96\r\n', 'DELE 48\r\n', 'DELE 95\r\n', 'DELE 110\r\n', 'DELE 12\r\n', 'DELE 27\r\n', 'DELE 56\r\n', 'DELE 34\r\n', 'DELE 55\r\n', 'DELE 132\r\n', 'DELE 38\r\n', 'DELE 26\r\n', 'DELE 86\r\n', 'DELE 115\r\n', 'DELE 103\r\n', 'DELE 17\r\n', 'DELE 101\r\n', 'DELE 8\r\n', 'DELE 11\r\n', 'DELE 33\r\n', 'DELE 53\r\n', 'DELE 88\r\n', 'DELE 76\r\n']
Please give 3 possible and different regular expressions to match all of the elements. 
You can give only 1 expression if the 3 expressions you find are too similar. 
Let’s work this out in a step-by-step way to make sure we have the right answer. 
To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\r\n’, ‘CAPA\r\n’, ‘\x15\x03\x01’, ‘GET / HTTP/1.0\r\n\r\n’, ‘r\n\r\n’] 
You only need to give me the three regular expressions in code format.
'''
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt,
)

response = model.generate_content(user_prompt)
print(response.text)

In [ ]:
def clean_json_string(s):
    # 使用正则表达式移除字符串前后的 '''json 和 '''
    cleaned_string = regex.sub(r"^'''json|'''$", '', s, flags=regex.MULTILINE).strip()
    return cleaned_string
output_string = clean_json_string(input_string)
output_string


### Evaluation Generated Regex
Randonly take 20 positive and 20 negateve data as input to generated regex and answer regex, comparing there result.

In [ ]:
#隨機選擇除了給定的SID以外的100個text
def select_random_texts(sid_to_unique_texts, given_sid, num_texts=100):
    # Filter out the given SID
    filtered_texts = [texts for sid, texts in sid_to_unique_texts.items() if sid != given_sid]
    
    # Flatten the list of lists to a single list of texts
    all_texts = [text for sublist in filtered_texts for text in sublist]
    
    # Randomly select 100 texts, or all texts if there are fewer than 100
    selected_texts = random.sample(all_texts, min(len(all_texts), num_texts))
    
    return selected_texts

def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                return row['pcre']
    return None

def match_patterns(targetText, GeneratedPatternList,isPositive=True):
    non_matching_patterns = []  # Step 1: Initialize list for non-matching patterns
    # Check each pattern and add non-matching ones to the list
    for i, pattern in enumerate(GeneratedPatternList):
        if not regex.search(pattern, targetText, regex.DOTALL):
            non_matching_patterns.append(f"Pattern {i+1}: {pattern}")
    ourResult = True if (9-len(non_matching_patterns)) >= 7 else False
    with open('evaluation_result.txt', 'w') as file:
        for pattern in non_matching_patterns:
            file.write(pattern)
    #ansMatch = bool(regex.search(ansPattern, targetText))
    return ourResult == isPositive

def positive_answer_evaluation(threeAnsPattern,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + bool(regex.search(ansPattern, text, regex.DOTALL))
        if not match_patterns(text, threeAnsPattern, True):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    print(f"errorList: {errorList}")
    return correct, total
def negative_answer_evaluation(GeneratedPatternList,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        if(not regex.search(ansPattern, text, regex.DOTALL)):
            correct = correct + 1
        if  not match_patterns(text, GeneratedPatternList, False):
            errorList.append(text)
    print("negative test")
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        print(error_text)
    print(f"errorList: {errorList}")
    
def positive_evaluation(threeAnsPattern,sid,file):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, threeAnsPattern, True)
        if not match_patterns(text, threeAnsPattern, True):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    file.write('print("positive")\n')
    for error in errorList:
        file.write('print(f"errorList: {errorList}")\n')
    return correct, total

def negative_evaluation(GeneratedPatternList,sid,file):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, GeneratedPatternList, False)
        if  not match_patterns(text, GeneratedPatternList, False):
            errorList.append(text)
    file.write('print("negative")\n')
    for error in errorList:
        file.write('print(f"errorList: {errorList}")\n')
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        
    
regex1 = r"/^([E|e][H|h][L|l][O|o])( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex2 = r"/^EHLO( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex3 = r"/^([Ee][Hh][Ll][Oo])( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex4 = r"^([E|e][H|h][L|l][O|o] [a-zA-Z0-9.-]+(\.[a-zA-Z]{2,})?\r\n)$"
regex5 = r"^([E|e][H|h][L|l][O|o] [a-zA-Z0-9.-]+(\.[a-zA-Z0-9.-]+)+\r\n|[E|e][H|h][L|l][O|o]\r\n)$"
regex6 = r"^([E|e][H|h][L|l][O|o]( [a-zA-Z0-9.-]+(\.[a-zA-Z0-9.-]+)+)?\r\n)$"
regex7 = r"^(EHLO|ehlo)( [^\r\n]+)?(\r\n)$"
regex8 = r"^([E|e][H|h][L|l][O|o])( [a-zA-Z0-9.-]+)?(\r\n)$"
regex9 = r"^([Ee][Hh][Ll][Oo])( [a-zA-Z0-9.-]+)?(\r\n)$"
GeneratedPatternList = [
    regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9
]

sid = '1454621'
positive_evaluation(GeneratedPatternList, sid)
negative_evaluation(GeneratedPatternList, sid)

### Helper Function

In [ ]:
def list_csv_columns():
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        # Print all column names
        print(reader.fieldnames)

In [ ]:
#print filtered_sids and corresponding text
for sid in filtered_sids:
    print(f"sid: {sid}, text: {sid_to_unique_texts[sid]}")

In [ ]:
protocols = [
    {'name': 'pop_df', 'data': pop_df},
    {'name': 'imap_df', 'data': imap_df},
    {'name': 'smtp_df', 'data': smtp_df},
    {'name': 'sip_df', 'data': sip_df}
]

with open('prompt.txt', 'w') as file:
    pass

with open('prompt.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        # Sort the filtered_sids to ensure the output is ordered by SID
        sorted_filtered_sids = sorted(filtered_sids)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in sorted_filtered_sids:
            texts = list(sid_to_unique_texts[sid])
            random.shuffle(texts)
            selected_texts = texts[:50]
            file.write(f"sid: {sid}, text: {selected_texts}\n")
            file.write("\n")
            file.write("Please find a regular expression to match all packet payloads.\n" + 
                       "You need to find the similarities in the sentences and generalize the parts where they differ. \n" + 
                        "The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. \n" + 
                        "There will be examples to help you find the patterns. \n"  +
                        "[‘DELE 3\\r\\n’, ‘DELE 128\\r\\n’, ‘DELE 74\\r\\n’, ‘DELE 22\\r\\n’, ‘DELE 70\\r\\n’] \n" +
                        "These examples show the attacker is trying to delete someone’s email by POP protocol. \n" +
                        "The index of the desired mail is indicated under the DELE command. \n" +
                        "Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\\r\\n)$’ \n" + "\n" +
                        "With the given example payloads: \n" +
                        "[‘EHLO BtuCBHdSb51.com\\r\\n’, ‘EHLO 203.187.87.27\\r\\n’, ‘EHLO slae02Fo9Ep.com\\r\\n’, ‘EHLO 210.64.37.51\\r\\n’, ‘EHLO LLb0RwqdbkikFWo.com\\r\\n’] \n" + 
                        "These examples show the attacker is trying to make sure the SMTP server is up and running. " + 
                        "The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. \n" +
                        "Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\\r\\n)$’ \n" + "\n" +
                        "Next, with the given payloads: \n")
            file.write(f"{selected_texts}\n")         
            file.write("Please give 3 possible and different regular expressions to match all of the elements. \n" +
                        "You can give only 1 expression if the 3 expressions you find are too similar. \n" + 
                        "Let’s work this out in a step-by-step way to make sure we have the right answer. \n" + 
                        "To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\\r\\n’, ‘CAPA\\r\\n’, ‘\\x15\\x03\\x01’, ‘GET / HTTP/1.0\\r\\n\\r\\n’, ‘r\\n\\r\\n’] \n" +
                        "You only need to give me the three regular expressions in code format.\n")
            file.write("\n")
        file.write("\n")

In [ ]:
with open('all_texts.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            file.write(f"sid: {sid}, text: {sid_to_unique_texts[sid]}\n")
                            
        file.write("\n")

In [ ]:
#獲取特定資料集的所有SID
def get_unique_sids_list(df):
    unique_sids = df[df['text'].notnull()]['sid'].unique()
    return unique_sids
sidList = get_unique_sids_list(pop_df)
print(sidList)
#print(len(sidList))